<a href="https://pymt.readthedocs.io"><img style="float: right" src="images/pymt-logo-header-text.png"></a>

# Running PyMT models

In this tutorial we will learn how to use *PyMT* to:
* Initialize models
* Advance models through time
* Get values from a model
* Unit conversion

We will also learn about the common interface for *PyMT* models.

We will run a single model in *pymt*. We will use the water balance
model, *Hydrotrend* (written in C). We'll setup a model simulation, run it, and then
analyze the output. You will also be given a chance to do your own simulations
and explore the HydroTrend model. 

* [Explore the base-case river simulation](#Exercise-1)
* [How does a river system respond to climate change?](#Exercise-2)

## HydroTrend

HydroTrend is a 2D hydrological water balance and transport model that simulates water
discharge and sediment load at a river outlet. You can read more about the model, find
references or download the source code at: https://csdms.colorado.edu/wiki/Model:HydroTrend.

### River Sediment Supply Modeling

This notebook is meant to give you a better understanding of what the model is capable of. In this
example we are using a theoretical river basin of ~1990 km<sup>2</sup>, with 1200m of relief and
a river length of ~100 km. All parameters that are shown by default once the HydroTrend Model
is loaded are based on a present-day, temperate climate. Whereas these runs are not meant to
be specific, we are using parameters that are realistic for the [Waiapaoa River][map_of_waiapaoa]
in New Zealand. The Waiapaoa River is located on North Island and receives high rain and has
erodible soils, so the river sediment loads are exceptionally high. It has been called the
*"dirtiest small river in the world"*.

To learn more about HydroTrend and its approach to sediment supply modeling, you can download
this [presentation][hydrotrend_presentation].

A more detailed description of applying HydroTrend to the Waipaoa basin, New Zealand has been published in WRR: [hydrotrend_waipaoa_paper]. 

[map_of_waiapaoa]: https://www.google.com/maps/place/Waipaoa+River/@-38.5099042,177.7668002,71814m/data=!3m1!1e3!4m5!3m4!1s0x6d65def908624859:0x2a00ef6165e1dfa0!8m2!3d-38.5392405!4d177.8843782
[hydrotrend_presentation]: https://csdms.colorado.edu/wiki/File:SedimentSupplyModeling02_2013.ppt
[hydrotrend_waipaoa_paper]: http://dx.doi.org/10.1029/2006WR005570

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Model initialization

With a set of input files, a model can be initialized, and then advanced through time. Upon model initialization, the input files are read, memory allocated and initialized so that time stepping can begin.

In [ ]:
from pymt import MODELS

hydrotrend = MODELS.Hydrotrend()
config_file, config_dir = hydrotrend.setup("hydrotrend-default")

hydrotrend.initialize(config_file, dir=config_dir)

## Input and output variables

Once initialized, we are able to obtain more information about the model, for instance names of input and output variables. The input variables, are names of variables that can be set dynamically as the model advances through time. Output variables are internal state variables that a model changes as it advances.

Input and output variables are accessed with the `input_var_names` and `output_var_names` attributes, respectively.

### Notice anything unusual?

In [ ]:
hydrotrend.input_var_names

In [ ]:
hydrotrend.output_var_names

Of this list of output variables, we're going to track water and sediment discharge, sediment
concentration and bedload flux. The mapping of these variables to the Standard Names
reported by *output_var_names* is given in the following table.

| Conventional Name      | Standard Name                                             |
| :--------------------- | :-------------------------------------------------------- |
| Water discharge        | channel_exit_water__volume_flow_rate                      |
| Sediment discharge     | channel_exit_water_sediment~suspended__mass_flow_rate     |
| Sediment concentration | channel_exit_water_sediment~suspended__mass_concentration |
| Bedload flux           | channel_exit_water_sediment~bedload__mass_flow_rate       |

### What do you think? Are you able to figure out what the other output variables are?

## Advancing a model through time

The attributes: `time_step`, `current_time`, `start_time`, `end_time`, and `time_units` provide information about how the model advances through time.

Using the above attributes, determine how many time steps have we set this model up to run for.

In [ ]:
start_time = hydrotrend.start_time
now = hydrotrend.time
end_time = hydrotrend.end_time
time_step = hydrotrend.time_step
time_units = hydrotrend.time_units

print(f"start time: {start_time} {time_units}")
print(f"current time: {now} {time_units}")
print(f"end time: {end_time} {time_units}")
print(f"time step: {time_step} {time_units}")

n_steps = (end_time - start_time) // time_step

print(f"number of time steps: {n_steps}")

We use the ***update*** method to advance *hydrotrend* one time step at a time and the ***get_value***
method to retreive values from the model. Feel free to add other variables if you like. Along the way, we'll save some variables to create a time series.

In [ ]:
n_steps = int(hydrotrend.end_time / hydrotrend.time_step)

q = np.empty(n_steps)
qs = np.empty(n_steps)
cs = np.empty(n_steps)
qb = np.empty(n_steps)

for i in tqdm(range(n_steps)):
    hydrotrend.update()
    
    q[i] = hydrotrend.get_value("channel_exit_water__volume_flow_rate")
    qs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_flow_rate")
    cs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_concentration")
    qb[i] = hydrotrend.get_value("channel_exit_water_sediment~bedload__mass_flow_rate")

Now plot the complete hydrograph.

In [ ]:
time = np.arange(len(qs))

plt.plot(time, qs)
plt.xlabel(f"Time ({time_units})")
plt.ylabel("Sediment discharge")

## Exercise 1

### Run for 100 years, calculate 100 year means

For this case study, we will run a simulation for 100 years at daily time-step.
This means you run Hydrotrend for 36,500 days total. Later on we will change
other input parameters but, for now, we'll just stick with the defaults.

Calculate mean water discharge Q, mean suspended load Qs, mean sediment concentration Cs, and mean bedload Qb.

*Note that all values are reported as daily averages. What are the units?*

In [ ]:
# Your code here

In [ ]:
from pymt import MODELS

hydrotrend = MODELS.Hydrotrend()
config_file, config_dir = hydrotrend.setup("hydrotrend-default", run_duration=100)

hydrotrend.initialize(config_file, dir=config_dir)

n_steps = int(hydrotrend.end_time / hydrotrend.time_step)

q = np.empty(n_steps)
qs = np.empty(n_steps)
cs = np.empty(n_steps)
qb = np.empty(n_steps)

for i in tqdm(range(n_steps)):
    hydrotrend.update()
    
    q[i] = hydrotrend.get_value("channel_exit_water__volume_flow_rate")
    qs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_flow_rate")
    cs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_concentration")
    qb[i] = hydrotrend.get_value("channel_exit_water_sediment~bedload__mass_flow_rate")

In [ ]:
(
    (q.mean(), hydrotrend.var_units("channel_exit_water__volume_flow_rate")),
    (cs.mean(), hydrotrend.var_units("channel_exit_water_sediment~suspended__mass_flow_rate")),
    (qs.mean(), hydrotrend.var_units("channel_exit_water_sediment~suspended__mass_concentration")),
    (qb.mean(), hydrotrend.var_units("channel_exit_water_sediment~bedload__mass_flow_rate"))
)

In [ ]:
hydrotrend.var_units("channel_exit_water__volume_flow_rate")

In [ ]:
print(f"River discharge: {q.mean()} m3 / s")
print(f"Suspended load: {qs.mean()} kg / s")
print(f"bedload: {qb.mean()} kg / s")

**Identify the highest flood event for this simulation. Is this the 50-year flood? Plot the year of Q-data which includes the flood.**


In [ ]:
# Your code here

In [ ]:
flood_day = q.argmax()
flood_year = flood_day // 365
plt.plot(q[flood_year * 365: (flood_year + 1) * 365])

In [ ]:
print(f"Year of max flood: {flood_year}")
print(f"Day of year of max flood: {flood_day % 365}")

## Exercise 2

### How does a river system respond to climate change? A few simple scenarios for the coming century.

#### What happens to river discharge, suspended load and bedload if the mean annual temperature in this specific river basin increases by 4 °C over the next 50 years?

In [ ]:
help(hydrotrend)

In [ ]:
from pymt import MODELS

hydrotrend = MODELS.Hydrotrend()

config_file, config_folder = hydrotrend.setup(run_duration=50, change_in_mean_annual_temperature=4.0 / 50.0)
hydrotrend.initialize(config_file, config_folder)

n_steps = int(hydrotrend.end_time / hydrotrend.time_step)

q = np.empty(n_steps)
qs = np.empty(n_steps)
cs = np.empty(n_steps)
qb = np.empty(n_steps)

for i in tqdm(range(n_steps)):
    hydrotrend.update()
    
    q[i] = hydrotrend.get_value("channel_exit_water__volume_flow_rate")
    qs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_flow_rate")
    cs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_concentration")
    qb[i] = hydrotrend.get_value("channel_exit_water_sediment~bedload__mass_flow_rate")

In [ ]:
(
    (q.mean(), hydrotrend.var_units("channel_exit_water__volume_flow_rate")),
    (qs.mean(), hydrotrend.var_units("channel_exit_water_sediment~suspended__mass_flow_rate")),
    (cs.mean(), hydrotrend.var_units("channel_exit_water_sediment~suspended__mass_concentration")),
    (qb.mean(), hydrotrend.var_units("channel_exit_water_sediment~bedload__mass_flow_rate"))
)

In [ ]:
print(f"River discharge: {q.mean()} m3 / s")
print(f"Suspended load: {qs.mean()} kg / s")
print(f"bedload: {qb.mean()} kg / s")

In [ ]:
# Your code here